In [1]:
import pickle
import pandas as pd
import numpy as np

countVector = pickle.load( open( "files/countVector.pickle", "rb" ) )


df = pickle.load( open( "files/df.pickle", "rb" ) )
df['Tweet'] = df['Tweet'].astype(str)
df['Len'] = df['Tweet'].str.split().apply(len)
df = df[df['Len']>=4]
print('tweets',df.shape[0])
print('Russian',np.sum(df['Category']))

tweets 8430264
Russian 2283309


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(countVector, df['Category'])

In [8]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
print(model.score(X_train, y_train), model.score(X_test, y_test))

0.9000434624585897 0.9001990922229719


In [9]:
pickle.dump( model, open( "files/model.pickle", "wb" ) )

In [4]:
model.coef_.shape

(1, 430)

In [5]:
def clean_text(text):
    # @somone to @
    text = ' '.join(['@' if '@' in a else a for a in text.split(' ')])
    # text = ' '.join(['#' if '#' in a else a for a in text.split(' ')])
    # remove all punctuation except @ and #
    text = ''.join([word.lower() for word in text if word not in string.punctuation.replace('@','').replace('#','')])
    # remove numbers
    text = re.sub('[0-9]+', 'number', text)
    tokens = nltk.word_tokenize(text)
    tokens2 = [a[0] if '@' in a[0] or '#' in a[0] else a[1] for a in nltk.pos_tag(text.split())]
    # hastags only for grammer-check, none in words
    tokens = [a.replace('#','') for a in tokens]
    tokens2 = ['#' if '#' in a else a for a in tokens2]
    lst = []
    lst = [ps.stem(word) for word in tokens]
    for i in range(4):
        for a in nltk.ngrams([ps.stem(word) for word in tokens2],i+3):
            lst.append(' '.join(a))
    
    return lst